In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, BatchNormalization, Dropout
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

# Load EEG dataset
file_path = "data/train.csv"
df = pd.read_csv(file_path)

# Relevant EEG features
relevant_columns = [
    "main.disorder",  # Target variable
    "AB.C.alpha.a.FP1", "AB.C.alpha.b.FP2", "AB.C.alpha.c.F7", "AB.C.alpha.d.F3",
    "AB.C.alpha.e.Fz", "AB.C.alpha.f.F4", "AB.C.alpha.g.F8", "AB.C.alpha.h.T3",
    "AB.C.alpha.i.C3", "AB.C.alpha.j.Cz", "AB.C.alpha.k.C4", "AB.C.alpha.l.T4",
    "AB.C.alpha.m.T5", "AB.C.alpha.n.P3", "AB.C.alpha.o.Pz", "AB.C.alpha.p.P4",
    "AB.C.alpha.q.T6", "AB.C.alpha.r.O1", "AB.C.alpha.s.O2",

    "COH.D.beta.a.FP1.b.FP2", "COH.D.beta.c.F7.d.F3", "COH.D.beta.e.Fz.f.F4",
    "COH.D.beta.g.F8.h.T3", "COH.D.beta.i.C3.j.Cz", "COH.D.beta.k.C4.l.T4",
    "COH.D.beta.m.T5.n.P3", "COH.D.beta.o.Pz.p.P4", "COH.D.beta.q.T6.r.O1",

    "AB.A.delta.a.FP1", "AB.A.delta.b.FP2", "AB.B.theta.a.FP1", "AB.B.theta.b.FP2",
    "AB.C.alpha.a.FP1", "AB.C.alpha.b.FP2", "AB.D.beta.a.FP1", "AB.D.beta.b.FP2",
    "AB.E.highbeta.a.FP1", "AB.E.highbeta.b.FP2", "AB.F.gamma.a.FP1", "AB.F.gamma.b.FP2",

    "COH.B.theta.a.FP1.b.FP2", "COH.B.theta.c.F7.d.F3", "COH.B.theta.e.Fz.f.F4",
    "COH.B.theta.g.F8.h.T3", "COH.B.theta.i.C3.j.Cz", "COH.B.theta.k.C4.l.T4",
    "COH.B.theta.m.T5.n.P3", "COH.B.theta.o.Pz.p.P4", "COH.B.theta.q.T6.r.O1",

    "AB.B.theta.a.FP1", "AB.B.theta.b.FP2", "AB.B.theta.c.F7", "AB.B.theta.d.F3",
    "AB.B.theta.e.Fz", "AB.B.theta.f.F4", "AB.B.theta.g.F8", "AB.B.theta.h.T3",
    "AB.B.theta.i.C3", "AB.B.theta.j.Cz", "AB.B.theta.k.C4", "AB.B.theta.l.T4",
    "AB.B.theta.m.T5", "AB.B.theta.n.P3", "AB.B.theta.o.Pz", "AB.B.theta.p.P4",
    "AB.B.theta.q.T6", "AB.B.theta.r.O1", "AB.B.theta.s.O2",

    "COH.F.gamma.a.FP1.b.FP2", "COH.F.gamma.c.F7.d.F3", "COH.F.gamma.e.Fz.f.F4",
    "COH.F.gamma.g.F8.h.T3", "COH.F.gamma.i.C3.j.Cz", "COH.F.gamma.k.C4.l.T4",
    "COH.F.gamma.m.T5.n.P3", "COH.F.gamma.o.Pz.p.P4", "COH.F.gamma.q.T6.r.O1"
]

df = df[relevant_columns]
# Target label
target_col = "main.disorder"

# Filter relevant columns & drop NaNs
df_eeg = df[[target_col] + relevant_columns].dropna()


In [2]:
scaler = MinMaxScaler()
df.iloc[:, 1:] = scaler.fit_transform(df.iloc[:, 1:])

# Separate Classes
classes = df["main.disorder"].unique()

synthetic_data = []

In [3]:
for disorder in classes:
    print(f"Training Autoencoder for: {disorder}")
    
    # Select data for current disorder
    df_class = df[df["main.disorder"] == disorder].drop(columns=["main.disorder"])
    
    # Split data
    X_train, X_val = train_test_split(df_class, test_size=0.2, random_state=42)

    # Define Autoencoder
    input_dim = X_train.shape[1]
    encoding_dim = 16  # Latent space size

    input_layer = Input(shape=(input_dim,))
    
    # Encoder
    encoded = Dense(64, activation="relu")(input_layer)
    encoded = BatchNormalization()(encoded)
    encoded = Dropout(0.2)(encoded)
    encoded = Dense(32, activation="relu")(encoded)
    encoded = Dense(encoding_dim, activation="relu")(encoded)

    # Decoder
    decoded = Dense(32, activation="relu")(encoded)
    decoded = Dense(64, activation="relu")(decoded)
    decoded = Dense(input_dim, activation="sigmoid")(decoded)

    autoencoder = Model(input_layer, decoded)
    autoencoder.compile(optimizer="adam", loss="mse")

    # Train
    autoencoder.fit(X_train, X_train,
                    epochs=500,
                    batch_size=128,
                    shuffle=True,
                    validation_data=(X_val, X_val),
                    verbose=1)

    # Generate Synthetic Data
    encoded_data = autoencoder.predict(X_train)
    synthetic_data.append(pd.DataFrame(encoded_data, columns=df_class.columns))

# Combine All Synthetic Data
synthetic_df = pd.concat(synthetic_data, axis=0, ignore_index=True)

# Check the correct number of rows per disorder
disorder_counts = [len(data) for data in synthetic_data]

# Debugging: Print to check counts
print("Expected counts per disorder:", disorder_counts)
print("Total synthetic rows:", synthetic_df.shape[0])
print("Number of disorders:", len(classes))

# Ensure `classes` is repeated to match the exact number of rows
synthetic_df["main.disorder"] = np.concatenate([
    np.full(count, disorder) for disorder, count in zip(classes, disorder_counts)
])

# Save the synthetic dataset
synthetic_df.to_csv("data/synthetic_eeg.csv", index=False)

print("Synthetic data successfully processed and saved!")
print(" Synthetic EEG data generated and saved.")

Training Autoencoder for: Mood disorder
Epoch 1/500
2/2 [==============================] - 2s 211ms/step - loss: 0.1454 - val_loss: 0.1404
Epoch 2/500
2/2 [==============================] - 0s 27ms/step - loss: 0.1426 - val_loss: 0.1390
Epoch 3/500
2/2 [==============================] - 0s 25ms/step - loss: 0.1400 - val_loss: 0.1374
Epoch 4/500
2/2 [==============================] - 0s 26ms/step - loss: 0.1371 - val_loss: 0.1354
Epoch 5/500
2/2 [==============================] - 0s 26ms/step - loss: 0.1339 - val_loss: 0.1327
Epoch 6/500
2/2 [==============================] - 0s 27ms/step - loss: 0.1302 - val_loss: 0.1292
Epoch 7/500
2/2 [==============================] - 0s 32ms/step - loss: 0.1255 - val_loss: 0.1249
Epoch 8/500
2/2 [==============================] - 0s 30ms/step - loss: 0.1201 - val_loss: 0.1194
Epoch 9/500
2/2 [==============================] - 0s 28ms/step - loss: 0.1139 - val_loss: 0.1129
Epoch 10/500
2/2 [==============================] - 0s 26ms/step - loss: 0.10